In [1]:
import sys 
import os
from __future__ import division
import numpy as np
from PIL import Image
import scipy as sc

In [4]:
sys.path.insert(0, '/home/jeetkanjani7/Tonbo/siamfc-tf/')

In [5]:
from src.tracker import tracker
import src.siamese as siam
from src.parse_arguments import parse_arguments
from src.region_to_bbox import region_to_bbox

Using Tensorflow 1.5.0


In [21]:
def init_video():
    video_folder = "/home/jeetkanjani7/Tonbo/siamfc-tf/data/dataset/tc_Ball_ce2/"
    frame_name_list = [f for f in os.listdir(video_folder) if f.endswith(".jpg")]
    frame_name_list = [ (video_folder) + s for s in frame_name_list] 
    frame_name_list.sort()
    with Image.open(frame_name_list[0]) as img:
        frame_sz = np.asanyarray(img.size)
        frame_sz[1], frame_sz[0] = frame_sz[0], frame_sz[1]
        
    gt_file = os.path.join(video_folder, 'groundtruth.txt')
    gt = np.genfromtxt(gt_file, delimiter = ',')
    n_frames = len(frame_name_list)
    assert n_frames == len(gt), 'Number of frames and number of GT lines should be equal'
    return gt, frame_name_list, frame_sz, n_frames 


In [9]:
def _compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    
    
    if xA < xB and yA < yB:
        interarea = (xB - xA) * (yB - yA)
        
        boxAArea = boxA[2] * boxA[3]
        boxBArea = boxB[2] * boxB[3]
        
        iou = interarea/(boxAArea + boxBArea - interarea)
    else:
        iou = 0
    assert iou >= 0
    assert iou<=1.01
    
    return iou

In [10]:
def _compute_distance(boxA, boxB):
    a = np.array((boxA[0]+boxA[2]/2, boxA[1]+boxA[3]/2))
    b = np.array((boxB[0]+boxB[2]/2, boxB[1]+boxB[3]/2))
    dist = np.linalg.norm(a - b)

    assert dist >= 0
    assert dist != float('Inf')

    return dist

In [11]:
def compile_results(gt, bboxes, dist_threshold):
    l = np.size(bboxes, 0)
    gt4 = np.zeros((l,4))
    new_distances = np.zeros(l)
    new_ious = np.zeros(l)
    n_thresholds = 50
    precisions_ths = np.zeros(n_thresholds)
    for i in range(l):
        gt4[i, :] = region_to_bbox(gt[i, :], center=False)
        new_distances[i] = _compute_distance(bboxes[i, :], gt4[i, :])
        new_ious[i] = _compute_iou(bboxes[i, :], gt4[i, :])
    precision = sum(new_distances < dist_threshold)/np.size(new_distances) * 100

    thresholds = np.linspace(0, 25, n_thresholds+1)
    thresholds = thresholds[-n_thresholds:]
    thresholds = thresholds[::-1]
    for i in range(n_thresholds):
        precisions_ths[i] = sum(new_distances < thresholds[i])/np.size(new_distances)

    precision_auc = np.trapz(precisions_ths)    
    iou = np.mean(new_ious) * 100

    return l, precision, precision_auc, iou

In [27]:
def main():
    hp, evaluation, run, env, design = parse_arguments()
    final_score_sz = hp.response_up * (design.score_sz -1) + 1
    filename, image, templates_z, scores = siam.build_tracking_graph(final_score_sz, design, env)
    print(scores)
    
        

In [28]:
main()

IOError: [Errno 2] No such file or directory: 'parameters/hyperparams.json'